In [ ]:


data_handler = DatasetHandler()
train_generator = data_handler.get_data_generator(train_data_path)
val_generator = data_handler.get_data_generator(test_data_path)

model_handler = ModelHandler(input_dim, hidden_dim, num_layers, output_dim)
model_handler.load_model()
model_handler.compile_model()
model_handler.train(train_generator, val_generator, n_epoch)


# 1.处理数据

In [69]:
import numpy as np
import tqdm
import itertools

from dataset import DKTDataSet
from torch.utils.data import DataLoader

from constants import *

class DatasetHandler():

    def __init__(self):
        pass

    def get_data_generator(self, data_path):
        ques, ans = self._read_data(data_path)
        dataset = DKTDataSet(ques, ans)
        data_generator = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
        return data_generator

    def _read_data(self, data_path):

        qus_list = np.array([])
        ans_list = np.array([])

        with open(data_path, 'r') as train:
            for num_seq, ques, ans in tqdm.tqdm(itertools.zip_longest(*[train] * 3), desc='loading data:    ', mininterval=2):

                num_seq = int(num_seq.strip().strip(','))
                ques = np.array(ques.strip().strip(',').split(',')).astype(int)
                ans = np.array(ans.strip().strip(',').split(',')).astype(int)

                mod = 0 if num_seq % MAX_SEQ == 0 else (MAX_SEQ - num_seq % MAX_SEQ)

                zero = np.zeros(mod) - 1
                ques = np.append(ques, zero)
                ans = np.append(ans, zero)

                qus_list = np.append(qus_list, ques)
                ans_list = np.append(ans_list, ans)

        qus_list = qus_list.astype(int).reshape([-1, MAX_SEQ])
        ans_list = ans_list.astype(int).reshape([-1, MAX_SEQ])

        return qus_list, ans_list

In [70]:
data_handler = DatasetHandler()
train_generator = data_handler.get_data_generator(train_data_path)

loading data:    : 0it [00:00, ?it/s]

loading data:    : 2154it [00:05, 390.40it/s]


In [71]:
train_generator#train_loder 

# 2.模型定义

In [72]:
import torch
import torch.nn as nn
from torch.autograd import Variable


#基于LSTM的DKT模型
class DKT(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(DKT, self).__init__()

        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.output_dim = output_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)#LSTM
        self.fc = nn.Linear(self.hidden_dim, self.output_dim)
        self.sig = nn.Sigmoid()

        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")#这里的作用？

    
    def _init_hidden(self, batch_size):
        """
        初始化隐藏状态和细胞状态
        Args:
            batch_size (int): 批次大小
        
        Returns:
            Tuple: 包含两个元素的元组，即(h_0, c_0)
        """
        h0 = Variable(torch.zeros(self.num_layers, batch_size, self.hidden_dim)).to(self.device)#这里其实不用Variable也可以，因为pytorch会自动转换
        c0 = Variable(torch.zeros(self.num_layers, batch_size, self.hidden_dim)).to(self.device)#细胞状态 是网络的一个核心组成部分，用于长期记忆信息的存储。它与隐藏状态一起协助网络在处理序列数据时维持重要信息并遗忘不必要的信息。细胞状态的设计使LSTM能够有效地解决传统循环神经网络（RNN）在长序列上训练时出现的梯度消失或梯度爆炸问题。
        # h0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(self.device)
        # c0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(self.device)

        # 也可以使用下面的方式初始化隐藏状态和细胞状态
        # weight = next(self.parameters()).detach()
        # h0 = weight.new(self.num_layers, batch_size, self.hidden_dim).zero_()
        # c0 = weight.new(self.num_layers, batch_size, self.hidden_dim).zero_()
        return h0, c0 

    def forward(self, x):
        batch_size = x.shape[0]
        h0, c0 = self._init_hidden(batch_size)
        out, hn = self.lstm(x, (h0.detach(), c0.detach()))#detach()的作用是什么？这意味着对于每个新的输入批次，LSTM的状态将不会考虑之前批次的梯度影响，确保每次前向传播都是从一个“干净”的状态开始的。
        res = self.sig(self.fc(out))

        return res

In [73]:
#损失函数定义
import torch
import torch.nn as nn

from constants import *

'''
Custom loss function for DKT
在 PyTorch 中，所有继承自 `nn.Module` 的类都自动拥有 `__call__()` 方法的特殊行为。这意味着任何自定义模块或损失函数，在被实例化后，可以像调用普通函数一样被调用，其实质是触发了 `__call__()` 方法的执行。下面详细解释这个机制：

### 如何工作
1. **`__call__` 方法的定义**：`nn.Module` 中的 `__call__` 方法封装了一些额外的步骤（如设置模块为训练或评估模式、执行前向钩子等），然后调用 `forward()` 方法。这是 PyTorch 框架中的一个关键设计，允许模块在执行其主要操作（即 `forward()` 中定义的操作）之前和之后自动执行预处理和后处理。

2. **直接调用实例**：当你创建一个继承自 `nn.Module` 的损失函数类的实例后，例如 `loss_fn = lossFunc()`，你可以通过 `loss = loss_fn(pred, batch)` 这样的语法直接调用这个实例。在这个调用发生时，Python 解释器会自动寻找并执行 `loss_fn` 的 `__call__` 方法。

3. **执行 `forward()`**：在 `__call__` 方法内部，PyTorch 会自动调用定义在损失函数类中的 `forward()` 方法，并传入相应的参数。因此，虽然你直接调用了实例，但实际执行的是 `forward()` 方法中定义的操作。

### 优势
这种设计有几个优点：
- **封装性**：用户只需要关心 `forward()` 方法中的逻辑，而不必担心调用时需要处理的其他底层细节。
- **灵活性**：通过钩子和其他机制，开发者可以在不修改主逻辑的情况下增加额外的功能。
- **一致性**：这种设计让所有的模块（无论是层、激活函数还是损失函数）都有相同的调用接口，简化了模型的设计和调试。

通过这种方式，PyTorch 简化了模型和组件的接口，使得开发和使用自定义模块变得更直接和一致。在实际应用中，这意味着你可以非常自然地将自定义损失函数融入任何模型架构中，而无需担心调用细节的差异。这种封装也使得代码更加整洁和易于维护。

'''
class lossFunc(nn.Module):#继承nn.Module，这是pytorch中定义神经网络的基类 为什么要这样做？https://blog.csdn.net/xholes/article/details/81413620 https://www.zhihu.com/question/66988664/answer/247952270 
    def __init__(self):
        super(lossFunc, self).__init__()

        self.loss_fn = nn.BCELoss()

    #只需要关心forward方法，不需要关心call方法
    def forward(self, pred, batch):#pred.shape = (batch, MAX_SEQ, QUESTION_NUM) batch.shape = (batch, MAX_SEQ, QUESTION_NUM*2)

        acc = 0#准确率初始化为0

        # qt 代表原始用户作答的问题序列，因为之前one-hot编码时，正确在前，错误在后，所以只需要取 前QUESTION_NUM个 +后QUESTION_NUM个，就得到了原始用户作答的问题序列 
        # def onehot(self, questions, answers):
        # result = np.zeros(shape=[MAX_SEQ, 2 * QUESTION_NUM])
        # for i in range(MAX_SEQ):
        #     if answers[i] > 0:
        #         result[i][questions[i]] = 1 
        #     elif answers[i] == 0:
        #         result[i][questions[i] + QUESTION_NUM] = 1
        # return result
        delta = batch[:,:,:QUESTION_NUM] + batch[:,:,QUESTION_NUM:] #delta [batch, MAX_SEQ, QUESTION_NUM] 

        # qt+1 & transpose for matrix multiplication
        delta = delta[:,1:,:].permute(0,2,1)#跳过第一个用户作答的问题序列，然后改变维度，变成(batch, QUESTION_NUM, MAX_SEQ-1)

        # yt 
        y = pred[:, :MAX_SEQ - 1,:]# y [batch, MAX_SEQ-1, QUESTION_NUM] 不包括最后一个预测结果

        # pred at+1
        temp = torch.matmul(y, delta) #temp是一个(batch, MAX_SEQ -1, MAX_SEQ-1)的矩阵，里面的值是概率值

        # get excercise prob from diagonal matrix
        prob = torch.diagonal(temp, dim1=1, dim2=2) # batch, MAX_SEQ-1(prob) 提取对角线元素，dim1=1, dim2=2代表从每一个 (D0) 的层（第一维）中，提取第二维和第三维形成的矩阵的对角线元素，最终形状是(batch, MAX_SEQ-1)，对于每一个层i（在第一维度），我们会得到一个向量prob
        '''
            为什么对角线元素代表每个时间步的预测概率？
            y的形状是(batch, MAX_SEQ-1, QUESTION_NUM)，代表除了最后一个时间步的每个时间步的预测。
            delta的形状是(batch, MAX_SEQ-1,QUESTION_NUM)，跳过第一时间步的用户作答，以适应预测结果的对应，然后通过premute变形成(batch, QUESTION_NUM, MAX_SEQ-1)。

            torch.matmul(y, delta)执行的是一个批次中所有预测时间步对所有问题（通过预测的QUESTION_NUM和答案的QUESTION_NUM进行匹配）的矩阵乘法。这个操作将每个预测向量与相应的答案向量进行点乘，计算每个预测对应的概率。
            结果矩阵temp的形状为[batch, MAX_SEQ - 1, MAX_SEQ - 1]。

            对角线元素的意义
            对角线元素：矩阵的对角线元素，即当dim1与dim2索引相同的元素，代表了同一时间步骤内预测向量与对应的答案向量的点乘结果。换句话说，对于每一个时间步，对角线上的元素表示该时间步预测正确的概率。
            非对角线元素：表示不同时间步骤之间的预测与答案向量的点乘结果，这在当前的任务中可能不具有实际的意义或用途。
        '''

        # at ex) *[1, 0] *[correct, false]
        a = ((batch[:,:,:QUESTION_NUM] - batch[:,:,QUESTION_NUM:]).sum(2) + 1) // 2#sum(2)其实是一种降维，因为每一行只会有一个值，其他都是0，这样再+1  -1 偏移量就会变成0 ，代表错误，但是原先的1变成了2 ，所以要除以2
        # at+1 ex) *[1, 0] *[correct, false]
        a = a[:, 1:] #跳过第一个用户作答的答案序列 a.shape = (batch, MAX_SEQ-1)

        loss = self.loss_fn(prob, a)#计算BCELoss prob是预测值，a是真实值 prob和a的shape都是(batch, MAX_SEQ-1)
        # print(prob.shape, a.shape)

        predict = torch.where(prob > 0.5, 1, 0)#预测值大于0.5为1，否则为0 ,predict=[batch, MAX_SEQ-1]

        acc = torch.logical_and(predict, a)#逻辑与 这里的acc是一个tensor，shape是(batch, MAX_SEQ-1) 里面的值是bool值，正确为True，错误为False
        acc = acc.sum() / torch.numel(acc) #计算准确率，除以总元素个数

        return loss, acc.item()#acc是一个tensor，需要转换成标量


In [74]:
for i, batch in enumerate(train_generator):
    print(batch.shape)
    break

torch.Size([128, 50, 1322])


In [75]:
(((batch[:,:,:QUESTION_NUM] - batch[:,:,QUESTION_NUM:]).sum(2) + 1) // 2).shape #sum(2)其实是一种降维，因为每一行只会有一个值，其他都是0，这样再+1  -1 偏移量就会变成0 ，代表错误，但是原先的1变成了2 ，所以要除以2

torch.Size([128, 50])

In [30]:
(batch[:,:,:QUESTION_NUM] - batch[:,:,QUESTION_NUM:]).shape 

torch.Size([128, 50, 661])

In [4]:
# 理解对角线元素
import torch
y = torch.Tensor([[0,0,1,0],[1,0,0,0],[0,1,0,0]])# 标志矩阵
prob = torch.rand(3,4)# 预测概率矩阵


In [14]:
y,y.shape
# (tensor([[0., 0., 1., 0.],
#          [1., 0., 0., 0.],
#          [0., 1., 0., 0.]]),
#  torch.Size([3, 4]))
prob, prob.shape
# (tensor([[0.3753, 0.7279, 0.2748, 0.9534],
#          [0.3327, 0.7226, 0.7337, 0.9276],
#          [0.5371, 0.0491, 0.2263, 0.0596]]),
#  torch.Size([3, 4]))
y_prob = torch.matmul(prob, y.permute(1,0))# 矩阵乘法
y_prob
# tensor([[0.2748, 0.3753, 0.7279],
#         [0.7337, 0.3327, 0.7226],
#         [0.2263, 0.5371, 0.0491]])
torch.diagonal(y_prob, dim1=0, dim2=1)# 提取对角线元素
#tensor([0.2748, 0.3327, 0.0491]) 就是y中1位置对应prob中的概率

'''
举例子理解
### 矩阵数据和形状
假设你提供的`y`和`prob`具有以下形状和内容：
- `y`: 一个表示答题情况的二进制矩阵，形状为 `[3, 4]`，其中每一行代表一个时间步的答题情况，具体到某个问题是否被回答（`1`表示被回答）。
- `prob`: 一个表示做题概率的矩阵，形状也为 `[3, 4]`，其中每个元素代表对应问题的做题概率。

### 矩阵乘法操作
执行`torch.matmul(prob, y.permute(1,0))`：
- `y.permute(1,0)`会将`y`的形状从`[3, 4]`转换为`[4, 3]`。转换后，`y`的列变成行，行变成列。
- 这种转换后，`prob`矩阵（形状为`[3, 4]`）与转置后的`y`矩阵（形状为`[4, 3]`）进行矩阵乘法。结果是形状为`[3, 3]`的矩阵，其中的每一个元素是`prob`中每一行与`y`中每一列的点积。

### 矩阵乘法结果
这个矩阵乘法操作的结果（记作`y_prob`）：
- 每个元素（假设为`y_prob[i, j]`）表示`prob`的第`i`行与`y`的第`j`列的点积。由于`y`的行现在代表问题，这个操作实质上是在计算每个问题在`prob`中的权重加和。

### 提取对角线元素
- `torch.diagonal(y_prob, dim1=0, dim2=1)`会从`y_prob`中提取对角线元素。这里的对角线元素代表每个问题在`prob`中的加权概率，其中权重是由`y`中的`1`来确定的。

### 为什么是做题位置的预测概率
在这个特定的示例中：
- `y`中的`1`代表某个问题被回答。
- 对角线上的值`[0.2748, 0.3327, 0.0491]`实际上代表了在每一个时间步中实际被回答的问题的预测概率。换句话说，这些对角线上的值是在每个时间步中，由`y`中的`1`确定的问题的预测概率。

通过这种方式，我们能够确切地知道在每个时间步中哪些问题被回答，以及这些问题的预测成功概率是多少。这为评估模型在预测每个具体问题的准确性提供了直接的手段。
'''

'\n举例子理解\n### 矩阵数据和形状\n假设你提供的`y`和`prob`具有以下形状和内容：\n- `y`: 一个表示答题情况的二进制矩阵，形状为 `[3, 4]`，其中每一行代表一个时间步的答题情况，具体到某个问题是否被回答（`1`表示被回答）。\n- `prob`: 一个表示做题概率的矩阵，形状也为 `[3, 4]`，其中每个元素代表对应问题的做题概率。\n\n### 矩阵乘法操作\n执行`torch.matmul(prob, y.permute(1,0))`：\n- `y.permute(1,0)`会将`y`的形状从`[3, 4]`转换为`[4, 3]`。转换后，`y`的列变成行，行变成列。\n- 这种转换后，`prob`矩阵（形状为`[3, 4]`）与转置后的`y`矩阵（形状为`[4, 3]`）进行矩阵乘法。结果是形状为`[3, 3]`的矩阵，其中的每一个元素是`prob`中每一行与`y`中每一列的点积。\n\n### 矩阵乘法结果\n这个矩阵乘法操作的结果（记作`y_prob`）：\n- 每个元素（假设为`y_prob[i, j]`）表示`prob`的第`i`行与`y`的第`j`列的点积。由于`y`的行现在代表问题，这个操作实质上是在计算每个问题在`prob`中的权重加和。\n\n### 提取对角线元素\n- `torch.diagonal(y_prob, dim1=0, dim2=1)`会从`y_prob`中提取对角线元素。这里的对角线元素代表每个问题在`prob`中的加权概率，其中权重是由`y`中的`1`来确定的。\n\n### 为什么是做题位置的预测概率\n在这个特定的示例中：\n- `y`中的`1`代表某个问题被回答。\n- 对角线上的值`[0.2748, 0.3327, 0.0491]`实际上代表了在每一个时间步中实际被回答的问题的预测概率。换句话说，这些对角线上的值是在每个时间步中，由`y`中的`1`确定的问题的预测概率。\n\n通过这种方式，我们能够确切地知道在每个时间步中哪些问题被回答，以及这些问题的预测成功概率是多少。这为评估模型在预测每个具体问题的准确性提供了直接的手段。\n\n'

In [10]:
prob, prob.shape

(tensor([[0.3753, 0.7279, 0.2748, 0.9534],
         [0.3327, 0.7226, 0.7337, 0.9276],
         [0.5371, 0.0491, 0.2263, 0.0596]]),
 torch.Size([3, 4]))

In [11]:
y_prob = torch.matmul(prob, y.permute(1,0))# 矩阵乘法
y_prob

tensor([[0.2748, 0.3753, 0.7279],
        [0.7337, 0.3327, 0.7226],
        [0.2263, 0.5371, 0.0491]])

In [57]:
torch.diagonal(y_prob, dim1=0, dim2=1)

tensor([0.2748, 0.3327, 0.0491])

In [23]:
#还有一种方法：nonzero()函数的作用是返回非零元素的索引
y = torch.Tensor([[[0,0,1,0],[1,0,0,0],[0,1,0,0]],[[0,0,1,0],[1,0,0,0],[0,1,0,0]]])# 标志矩阵
prob=torch.stack((torch.rand(3,4),torch.rand(3,4)))# 预测概率矩阵


tensor([0, 1, 2, 0, 1, 2])

In [60]:
y.nonzero()[:,2:3]

tensor([[2],
        [0],
        [1],
        [2],
        [0],
        [1]])

In [40]:
index=y.nonzero(as_tuple=True)#返回一个元组，元组中每个张量表示对应维度上所有非零元素的索引。
# (tensor([0, 0, 0, 1, 1, 1]),  # 第一个维度的索引
#  tensor([0, 1, 2, 0, 1, 2]),  # 第二个维度的索引
#  tensor([2, 0, 1, 2, 0, 1]))  # 第三个维度的索引
index#竖着看，代表第0个dim0，第0个dim1，第2个dim2有一个非0


(tensor([0, 0, 0, 1, 1, 1]),
 tensor([0, 1, 2, 0, 1, 2]),
 tensor([2, 0, 1, 2, 0, 1]))

In [50]:
selected_probabilities = prob[y.nonzero(as_tuple=True)]
selected_probabilities

tensor([0.8420, 0.5595, 0.8777, 0.6458, 0.8813, 0.9246])

In [25]:
y,prob

(tensor([[[0., 0., 1., 0.],
          [1., 0., 0., 0.],
          [0., 1., 0., 0.]],
 
         [[0., 0., 1., 0.],
          [1., 0., 0., 0.],
          [0., 1., 0., 0.]]]),
 tensor([[[0.9444, 0.3940, 0.8420, 0.2050],
          [0.5595, 0.9600, 0.1424, 0.9065],
          [0.2559, 0.8777, 0.0454, 0.7939]],
 
         [[0.4044, 0.8184, 0.6458, 0.9424],
          [0.8813, 0.7034, 0.5766, 0.4903],
          [0.2482, 0.9246, 0.6489, 0.2364]]]))

In [56]:
# 使用where
selected_probabilities = torch.where(y == 1, prob, torch.tensor(0.0))
selected_probabilities.sum(2).flatten()

tensor([0.8420, 0.5595, 0.8777, 0.6458, 0.8813, 0.9246])

# 3. 训练逻辑

In [86]:
import torch
import torch.optim as optim

from models.DKT import DKT
from loss import lossFunc

import tqdm
import time
import os

from constants import *

#模型处理类，里面包含了模型的训练、验证、测试等方法
class ModelHandler():

    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.output_dim = output_dim

        self.model = None
        self.loss_fn = None
        self.optimizer = None

        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        if not os.path.isdir('outputs'):#outputs存放pth文件，pth是pytorch模型的保存格式
            os.mkdir('outputs')

        print('-' * 30)
        print('使用GPU-' * 10, self.device, '-' * 10)
        print('-' * 30)

        pass

    def load_model(self, model_path=None):
        if model_path == None:#如果没有传入模型路径，则初始化一个新的模型
            self.model = DKT(self.input_dim, self.hidden_dim, self.num_layers, self.output_dim)
        else:#如果传入了模型路径，则加载模型
            self.model = DKT(self.input_dim, self.hidden_dim, self.num_layers, self.output_dim)
            self.model.load_state_dict(torch.load(model_path))#加载模型参数
        pass

    
    def compile_model(self):
        '''
        编译模型，定义优化器和损失函数
        '''
        self.optimizer = optim.Adam(self.model.parameters(), lr=LR)
        self.loss_fn = lossFunc().to(self.device)#损失函数是自定义的，这里是调用的lossFunc类，为什么要调用to(self.device)？
        self.model.to(self.device)#将模型放到GPU上
        
        print('compile model', '-' * 20)
        print(self.optimizer)
        print('')
        print(self.loss_fn)
        print('')
        print(self.model)
        print('-' * 30)

    #保存模型，主要是保存模型参数
    def save_model(self, epoch, val_loss, val_acc):

        save_path = f'outputs/{epoch}_{val_loss:.4f}_{val_acc:.4f}.pth'
        torch.save(self.model.state_dict(), save_path)

        pass

    #训练模型
    def train(self, train_generator, val_generator, n_epoch):

        min_loss = 10e+8#初始化最小损失为一个很大的数 ，这样可以保证第一次验证的损失一定小于这个数

        for epoch in tqdm.tqdm(range(n_epoch), desc='Training:', mininterval=2):#tqdm需要传入一个可迭代对象，这里传入了range(n_epoch)，表示迭代n_epoch次

            # training step
            running_loss, running_acc = self._optimize(train_generator, epoch, train=True)
            print(f"epoch : {epoch}/{n_epoch}  running_acc : {running_acc:.4f}, running_loss : {running_loss.item():.4f}")#打印训练过程中的损失和准确率

            # validation step
            if epoch % 5 == 0:#每5个epoch进行一次验证
                with torch.no_grad():#关闭梯度计算
                    self.model.eval()#模型进入验证模式
                    val_loss, val_acc = self._optimize(val_generator, epoch, train=False)
                    print(f"epoch : {epoch}/{n_epoch}  val_acc : {val_acc:.4f}, val_loss : {val_loss.item():.4f}")#打印验证过程中的损失和准确率

                if val_loss < min_loss:#如果验证损失小于最小损失，则保存模型，只保存每次验证损失最小的模型，这样就实现了最优模型的保存？
                    min_loss = val_loss#更新最小损失
                    self.save_model(epoch, min_loss, val_acc)
        pass
    
    #分类处理练和验证
    def _optimize(self, data_generator, epoch, train=True):

        start = time.time()#记录时间

        if train:

            running_loss = 0
            running_acc = 0

            self.model.train()#模型进入训练模式

            for num, batch in enumerate(data_generator):#num是批次的序号，batch是一个批次的数据 batch.shape = [batch_size, seq_len, input_dim=QUESTION_NUM * 2]

                batch = batch.to(self.device)#将数据放到GPU上,需要手动放到GPU上

                # wipe any existing gradients from previous iterations
                self.optimizer.zero_grad()#梯度清零，因为pytorch默认会将梯度累加

                # forward pass
                pred = self.model(batch)

                # compute the loss and accuracy 
                loss, acc = self.loss_fn(pred, batch)

                # this step computes all gradients with "autograd"
                # i.e. automatic differentiation
                loss.backward()

                # this actually changes the parameters
                self.optimizer.step()

                # if the current loss is better than any ones we've seen
                # before, save the parameters.

                running_loss += loss#累加损失
                running_acc += acc#累加准确率

                end = time.time()#记录时间

                if (num + 1) % 16 == 0: # print every 16 mini-batches，只查看
                    print(#len(data_generator)是一个epoch内的批次个数
                        f"[{epoch} epoch {num + 1}/{len(data_generator)} iter] batch_running_acc : {acc:.4f}, batch_running_loss : {loss.item():.4f} time : {end - start:.2f} sec",
                        end='\r', flush=True)#打印every 16 mini-batches批次的损失和准确率，flush=True表示立即刷新输出

            running_loss = running_loss / len(data_generator)#计算平均损失，这个平均是在一个epoch内的平均
            running_acc = running_acc / len(data_generator)

            return running_loss, running_acc

        else:

            val_loss = 0
            val_acc = 0

            self.model.eval()

            for num, batch in enumerate(data_generator):

                batch = batch.to(self.device)

                with torch.no_grad():

                    pred = self.model(batch)
                    loss, acc = self.loss_fn(pred, batch)
                    val_loss += loss
                    val_acc += acc

                    end = time.time()

                if num % 16 == 1:
                    print(
                        f"[{epoch + 1} epoch {num + 1}/{len(data_generator)} iter] batch_val_acc : {acc:.4f}, batch_val_loss : {loss.item():.4f} time : {end - start:.2f} sec",
                        end='\r', flush=True)

            val_loss = val_loss / len(data_generator)
            val_acc = val_acc / len(data_generator)

            return val_loss, val_acc

    #评估模型
    def evaluate(self, test_generator):
        test_loss, test_acc = self._optimize(test_generator, 0, train=False)
        print('-'*50)
        print(f" test_acc : {test_acc:.4f}, test_loss : {test_loss.item():.4f}")
        pass

    #预测
    def predict(self, x):
        def _cal_prob(x):

            # qt
            delta = x[:,:,:QUESTION_NUM] + x[:,:,QUESTION_NUM:]

            # qt+1
            delta = delta[:,1:,:].permute(0,2,1)

            # yt
            pred = self.model(x)
            y = pred[:, :MAX_SEQ - 1,:]

            # pred at+1
            temp = torch.matmul(y, delta) # 1, MAX_SEQ, MAX_SEQ-1(prob)

            # get excercise prob from diagonal matrix
            prob = torch.diagonal(temp, dim1=1, dim2=2) # 1, MAX_SEQ-1(prob)\

            return prob.squeeze(0)#去掉维度为1的维度

        # get excercise tag sequence
        def _get_q_sequence(q_seq_one_hot):#q_seq_one_hot.shape = [1, MAX_SEQ, QUESTION_NUM * 2]

            q_sequence = []
            one_hot_excercise_tags = q_seq_one_hot[:, :, :QUESTION_NUM] + q_seq_one_hot[:, :, QUESTION_NUM:]
            one_hot_excercise_tags = one_hot_excercise_tags.squeeze(0)#去掉维度为1的维度

            for one_hot_excercise_tag in one_hot_excercise_tags:
                try:
                    excercise_tag = torch.nonzero(one_hot_excercise_tag).item()
                except:
                    excercise_tag = -1

                q_sequence.append(excercise_tag)

            return torch.Tensor(q_sequence)
        # get answer sequence
        def _get_a_sequence(q_seq_one_hot):
            q_seq_one_hot = q_seq_one_hot.squeeze(0)#去掉维度为1的维度
            a_sequence = ((q_seq_one_hot[:, :QUESTION_NUM] - q_seq_one_hot[:, QUESTION_NUM:]).sum(1) + 1) // 2
            return a_sequence


        if len(x.size()) == 2:#如果输入的数据维度是2，则增加一个维度,因为模型的输入是3维的,增加的维度是batch_size
            x = x.unsqueeze(0)

        x = x.to(self.device)#将数据放到GPU上

        prob = _cal_prob(x)#计算预测概率，prob.shape = [MAX_SEQ-1]

        q_sequence = _get_q_sequence(x)
        a_sequence = _get_a_sequence(x)

        if -1 in q_sequence:#如果有-1，代表这个学生的问题序列已经结束了，所以只需要取到-1之前的问题序列
            last_excercise_tag = torch.nonzero(q_sequence == -1)[0][0].item() - 1
        else:
            last_excercise_tag = len(q_sequence) - 1

        print('-' * 50)
        print(f'sol excercise tags: \n {q_sequence[:last_excercise_tag-1]}')
        print(f'result excercise tags: \n {a_sequence[:last_excercise_tag-1]}')
        print('-' * 50)

        print(f'predict excercise tag {q_sequence[last_excercise_tag]}')
        print(f'ground truth : {a_sequence[last_excercise_tag]}')
        print(f'this student has a {prob[last_excercise_tag-1]*100:.2f}% chance of solving this problem')


In [83]:
prob=prob.sum(2).flatten()

In [85]:
prob.squeeze(0)

tensor([0.8420, 0.5595, 0.8777, 0.6458, 0.8813, 0.9246])